In [88]:
import sys
sys.path.append('/home/jaxmao/jaxmao_branches/JaxMao/')
sys.path.append('/home/jaxmao/jaxmao_branches/JaxMao/jaxmao')

import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import jax
import jax.numpy as jnp
import numpy as np

import jaxmao
from jaxmao.layers import Conv2D, SimpleDense, Dense, BatchNorm, ReLU, Flatten, StableSoftmax, BatchNorm2D, DepthwiseConv2D, Activation
from jaxmao.modules import Module
from jaxmao.optimizers import GradientDescent
from jaxmao.losses import CategoricalCrossEntropy
from jaxmao.metrics import Accuracy, Precision, Recall

print('jax.devices() :', jax.devices())

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
print('tf.config.list_physical_devices(): ', tf.config.list_physical_devices())

seed = 42
key = jax.random.PRNGKey(seed)

tf.keras.backend.set_floatx('float32')

jax.devices() : [CpuDevice(id=0)]
tf.config.list_physical_devices():  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [89]:
def close_enough(A, B, eps=1e-10):
    return np.less_equal(np.abs(A - B), eps)

# SimpleDense

In [90]:
# generate data
sample_data = np.random.normal(0, 1, (8, 16))

# JaxMao
jaxmao_dense = jaxmao.layers.SimpleDense(16, 4, 'linear')
jaxmao_dense.init_params(key)
jaxmao_predicted = jaxmao_dense(sample_data)

# TensorFlow
tf_dense = keras.layers.Dense(4)
tf_dense.build(input_shape=(16, ))
tf_dense.set_weights([jaxmao_dense.params['weights'], jaxmao_dense.params['biases']])
tf_dense = Sequential([tf_dense])
tf_predicted = tf_dense.predict(sample_data)

print(close_enough(tf_predicted, jaxmao_predicted))

1/1 [==============================] - 0s 44ms/step
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


# Softmax
different implementation of softmax

In [91]:
# generate data
sample_data = np.random.normal(0, 1, (8, 16))

# JaxMao
jaxmao_dense = jaxmao.layers.SimpleDense(16, 4, 'softmax')
jaxmao_dense.init_params(key)
jaxmao_predicted = jaxmao_dense(sample_data)

# TensorFlow
tf_dense = keras.layers.Dense(4, activation='softmax')
tf_dense.build(input_shape=(16, ))
tf_dense.set_weights([jaxmao_dense.params['weights'], jaxmao_dense.params['biases']])
tf_dense = Sequential([tf_dense])
tf_predicted = tf_dense.predict(sample_data)

print(close_enough(tf_predicted, jaxmao_predicted, 1e-7)) 

1/1 [==============================] - 0s 60ms/step
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


# relu

In [92]:
# generate data
sample_data = np.random.normal(0, 1, (8, 16))

# JaxMao
jaxmao_dense = jaxmao.layers.SimpleDense(16, 4, 'relu')
jaxmao_dense.init_params(key)
jaxmao_predicted = jaxmao_dense(sample_data)

# TensorFlow
tf_dense = keras.layers.Dense(4, activation='relu')
tf_dense.build(input_shape=(16, ))
tf_dense.set_weights([jaxmao_dense.params['weights'], jaxmao_dense.params['biases']])
tf_sequential = Sequential([tf_dense])
tf_predicted = tf_sequential.predict(sample_data)

print(close_enough(tf_predicted, jaxmao_predicted))

1/1 [==============================] - 0s 67ms/step
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


# BatchNorm (inference)
only consistently tolerate up to 1e-6.
not sure why

In [93]:
# Generate random 1D data
sample_data = np.random.normal(0, 1, (8, 16))  # shape (batch_size, features)

# JaxMao BatchNorm1D
jaxmao_batchnorm = jaxmao.layers.BatchNorm(16, eps=1e-10)
jaxmao_batchnorm.init_params(key)
jaxmao_batchnorm.switch_mode('inference')
jaxmao_batchnorm_output = jaxmao_batchnorm(sample_data)

# TensorFlow BatchNormalization
tf_batchnorm = keras.layers.BatchNormalization(axis=-1, epsilon=1e-10)
tf_batchnorm.build(input_shape=(None, 16))
tf_batchnorm.set_weights([
    jaxmao_batchnorm.params['gamma'], 
    jaxmao_batchnorm.params['beta'], 
    jaxmao_batchnorm.state['running_mean'], 
    jaxmao_batchnorm.state['running_var']
])
tf_batchnorm_output = tf_batchnorm(sample_data, training=False).numpy()

# Compare
print(close_enough(jaxmao_batchnorm_output, tf_batchnorm_output, 1e-6))

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]


# batchnorm (training)
same with inference. only tolerate up to 1e-6

In [94]:
# Generate random 1D data
sample_data = np.random.normal(0, 1, (8, 16))  # shape (batch_size, features)

# JaxMao BatchNorm1D
jaxmao_batchnorm = jaxmao.layers.BatchNorm(16, eps=1e-8)
jaxmao_batchnorm.init_params(key)
jaxmao_batchnorm.switch_mode('train')
jaxmao_batchnorm_output, new_state = jaxmao_batchnorm.pure_forward(jaxmao_batchnorm.params, sample_data, jaxmao_batchnorm.state)
jaxmao_batchnorm.update_state(new_state)

# TensorFlow BatchNormalization
tf_batchnorm = keras.layers.BatchNormalization(axis=-1, epsilon=1e-8)
tf_batchnorm.build(input_shape=(None, 16))
tf_batchnorm.set_weights([
    jaxmao_batchnorm.params['gamma'], 
    jaxmao_batchnorm.params['beta'], 
    jaxmao_batchnorm.state['running_mean'], 
    jaxmao_batchnorm.state['running_var']
])
tf_batchnorm_output = tf_batchnorm(sample_data, training=True).numpy()

# Compare
print(close_enough(jaxmao_batchnorm_output, tf_batchnorm_output, 1e-6))

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]


# dense relu bn

In [142]:
# generate data
batch_size = 8
channels = 5

sample_data = np.zeros((batch_size, channels))

for c in range(channels):
    mean = np.random.randint(-np.random.randint(-12, 2), np.random.randint(42, 52)) 
    std_dev = np.random.uniform(-2, 421) 
    sample_data[ :, c] = np.random.normal(mean, std_dev, (batch_size, ))

# JaxMao
jaxmao_dense = jaxmao.layers.SimpleDense(5, 4, 'relu', use_bias=False)
jaxmao_bn = jaxmao.layers.BatchNorm1D(4, eps=1e-5)
jaxmao_dense.init_params(key)
jaxmao_bn.init_params(key)
jaxmao_bn.switch_mode('inference')

z1 = jaxmao_dense(sample_data)
jaxmao_predicted = jaxmao_bn(jaxmao_predicted)

# TensorFlow
tf_dense = keras.layers.Dense(4, activation='relu', use_bias=False)
tf_bn = keras.layers.BatchNormalization(epsilon=1e-5)
tf_dense.build(input_shape=(None, 5))
tf_dense.set_weights([jaxmao_dense.params['weights']])

z2 = tf_dense(sample_data)
tf_predicted = tf_bn(tf_dense(sample_data), training=False)

print('before bn \n', z1 == z2)
print('\nafter bn \n', close_enough(tf_predicted, jaxmao_predicted, 1e-3))

before bn 
 tf.Tensor(
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]], shape=(8, 4), dtype=bool)

after bn 
 [[ True False False False]
 [False False False False]
 [False False False False]
 [False False False  True]
 [ True False False False]
 [False False False False]
 [False False  True  True]
 [False False False False]]


In [140]:
jaxmao_predicted - tf_predicted

<tf.Tensor: shape=(8, 4), dtype=float32, numpy=
array([[ 0.0000000e+00, -8.1704483e+01, -2.6055374e+02, -1.7759938e+02],
       [-2.9857681e+02,  0.0000000e+00,  5.0622451e-01, -2.1900783e+02],
       [-1.3072346e+02,  3.9482296e+00,  1.4341803e+00,  0.0000000e+00],
       [-4.1106900e+02, -2.4140935e+02, -4.1260101e+02, -5.7572498e+02],
       [ 0.0000000e+00,  0.0000000e+00, -2.4298613e+01, -3.0007215e+01],
       [ 3.5409102e+00,  0.0000000e+00,  0.0000000e+00,  1.3800922e-01],
       [-5.9365485e+02,  1.5038968e+00,  0.0000000e+00, -3.2684772e+02],
       [-4.7047058e+02, -5.6599331e+01,  4.5537469e-01, -2.7459564e+02]],
      dtype=float32)>

##### alternative in jaxmao

In [ ]:
# JaxMao
jaxmao_dense = jaxmao.layers.Dense(5, 4, 'softmax', batch_norm=True)
jaxmao_dense.init_params(key)
jaxmao_dense.switch_mode('inference')
jaxmao_predicted, new_state = jaxmao_dense.pure_forward(jaxmao_dense.params, 
                                                        sample_data, 
                                                        jaxmao_dense.state)

# TensorFlow
tf_dense = keras.layers.Dense(4, use_bias=False)
tf_bn = keras.layers.BatchNormalization()
tf_dense.build(input_shape=(5, ))
tf_dense.set_weights([jaxmao_dense.params['simple_dense']['weights']])
tf_sequential = Sequential([tf_dense, tf_bn, keras.layers.Activation('softmax')])
tf_predicted = tf_sequential.predict(sample_data)

print(close_enough(tf_predicted, jaxmao_predicted, 1e-3))

1/1 [==============================] - 0s 107ms/step
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


#### training mode

In [ ]:
# JaxMao
jaxmao_dense = jaxmao.layers.Dense(5, 4, 'linear', batch_norm=True)
jaxmao_dense.init_params(key)
jaxmao_dense.switch_mode('train')

# TensorFlow
tf_dense = keras.layers.Dense(4, use_bias=False)
tf_bn = keras.layers.BatchNormalization()
tf_dense.build(input_shape=(5, ))
tf_dense.set_weights([jaxmao_dense.params['simple_dense']['weights']])

for _ in range(10):
    jaxmao_predicted, new_state = jaxmao_dense.pure_forward(jaxmao_dense.params, 
                                                        sample_data, 
                                                        jaxmao_dense.state)
    jaxmao_dense.update_state(new_state)
    tf_z = tf_dense(sample_data)
    tf_predicted = tf_bn(tf_z, training=True)

print(close_enough(tf_predicted, jaxmao_predicted, 1e-6))

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


# convolution
also only consistent at eps=1e-6

In [ ]:
# generate data
sample_data = np.random.normal(0, 1, (8, 16, 16, 5))

# JaxMao
jaxmao_conv = jaxmao.layers.SimpleConv2D(5, 4, (3, 3), activation='relu', padding='SAME')
jaxmao_conv.init_params(key)
jaxmao_predicted = jaxmao_conv(sample_data)

# TensorFlow
tf_conv = keras.layers.Conv2D(4, (3,3), activation='relu', padding='same')
tf_conv.build(input_shape=(16, 16, 5))
tf_conv.set_weights([jaxmao_conv.params['weights'], jaxmao_conv.params['biases']])
tf_predicted = tf_conv(sample_data).numpy()

print(close_enough(tf_predicted, jaxmao_predicted, 1e-6).all())

True


# BatchNorm2d

In [144]:
# generate data, if use random.normal mean and var wouldn't be tested.
# Initialize shape parameters
batch_size = 8
height = 16
width = 16
channels = 5

# Initialize empty array
sample_image = np.zeros((batch_size, height, width, channels))

# Populate each channel with different mean and std dev
for c in range(channels):
    mean = np.random.randint(-np.random.randint(-12, 2), np.random.randint(42, 52)) 
    std_dev = np.random.uniform(-2, 421) 
    sample_image[:, :, :, c] = np.random.normal(mean, std_dev, (batch_size, height, width))

# JaxMao BatchNorm2D
jaxmao_batchnorm2d = jaxmao.layers.BatchNorm2D(channels, eps=1e-10, momentum=0.99)
jaxmao_batchnorm2d.init_params(key)
jaxmao_batchnorm2d.switch_mode('inference')
jaxmao_batchnorm2d_output = jaxmao_batchnorm2d(sample_image)

# TensorFlow BatchNormalization (2D)
tf_batchnorm2d = keras.layers.BatchNormalization(epsilon=1e-10)
tf_batchnorm2d.build(input_shape=(None, height, width, channels))
tf_batchnorm2d.set_weights([
    jaxmao_batchnorm2d.params['gamma'], 
    jaxmao_batchnorm2d.params['beta'], 
    jaxmao_batchnorm2d.state['running_mean'], 
    jaxmao_batchnorm2d.state['running_var']
])
tf_batchnorm2d_output = tf_batchnorm2d(sample_image, training=False).numpy()

# Compare
print(close_enough(jaxmao_batchnorm2d_output, tf_batchnorm2d_output, 1e-10).all())

True


In [148]:
# Generate random 2D data (image)
sample_image = np.random.normal(0, 1, (1, 8, 8, 6))  # shape (batch_size, height, width, channels)

# JaxMao BatchNorm2D
jaxmao_batchnorm2d = jaxmao.layers.BatchNorm2D(6)
jaxmao_batchnorm2d.init_params(key)
jaxmao_batchnorm2d.switch_mode('train')
jaxmao_batchnorm2d_output, state = jaxmao_batchnorm2d.pure_forward(jaxmao_batchnorm2d.params, sample_image, jaxmao_batchnorm2d.state)

# TensorFlow BatchNormalization (2D)
tf_batchnorm2d = keras.layers.BatchNormalization()
tf_batchnorm2d.build(input_shape=(None, 8, 8, 6))
tf_batchnorm2d.set_weights([
    jaxmao_batchnorm2d.params['gamma'], 
    jaxmao_batchnorm2d.params['beta'], 
    jaxmao_batchnorm2d.state['running_mean'], 
    jaxmao_batchnorm2d.state['running_var']
])
tf_batchnorm2d_output = tf_batchnorm2d(sample_image, training=True).numpy()

# Compare
print(close_enough(jaxmao_batchnorm2d_output, tf_batchnorm2d_output, 1e-6))

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True 

# convolution bn

In [149]:
class BatchNorm2D(BatchNorm):
    def __init__(
        self,
        num_features,
        momentum = 0.99,
        eps=1e-5
        ):
        super().__init__(num_features=num_features, momentum=momentum, axis_mean=(1, 2, 3), eps=eps)

In [150]:
# generate data
# Initialize shape parameters
batch_size = 8
height = 16
width = 16
channels = 5

# Initialize empty array
sample_data = np.zeros((batch_size, height, width, channels))

# Populate each channel with different mean and std dev
for c in range(channels):
    mean = np.random.randint(-np.random.randint(-12, 2), np.random.randint(42, 52),) 
    std_dev = np.random.uniform(-2, 421) 
    sample_data[:, :, :, c] = np.random.normal(mean, std_dev, (batch_size, height, width))


# JaxMao
jaxmao_conv = jaxmao.layers.Conv2D(5, 4, (3, 3), activation='linear', padding='SAME', batch_norm=True, use_bias=False)
jaxmao_conv.init_params(key)
jaxmao_conv.set_inference_mode()
# jaxmao_conv.set_training_mode()

jaxmao_predicted, state = jaxmao_conv(jaxmao_conv.params, sample_data)

# TensorFlow
tf_conv = keras.layers.Conv2D(4, (3,3), padding='same', use_bias=False)
tf_bn = keras.layers.BatchNormalization(epsilon=1e-5) # just realize that we can do this.
tf_conv.build(input_shape=(16, 16, 5))
tf_conv.set_weights([jaxmao_conv.params['conv2d/simple_conv2d']['weights']])

z = tf_conv(sample_data)
z = tf_bn(z, training=False)
tf_predicted = keras.layers.Activation('linear')(z)

print(close_enough(tf_predicted, jaxmao_predicted, 1e-5).all())

AttributeError: 'Conv2D' object has no attribute 'set_inference_mode'

# other layers ... 

# MaxPooling2d

In [ ]:
sample_image = np.random.normal(0, 1, (5, 8, 8, 3)) 

# JaxMao
jaxmao_maxpool = jaxmao.layers.MaxPooling2D(kernel_size=(2, 2), strides=(2, 2), padding='SAME')
jaxmao_maxpool.init_params(key)
jaxmao_pooled, state = jaxmao_maxpool(jaxmao_maxpool.params, sample_image)

# TensorFlow
tf_maxpool = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
tf_pooled = tf_maxpool(sample_image).numpy() 

# Compare
print(close_enough(jaxmao_pooled, tf_pooled).all())

True


# AveragePooling2d

In [ ]:
# Generate random image data
sample_image = np.random.normal(0, 1, (4, 8, 8, 3))  # shape (batch_size, height, width, channels)

# JaxMao AveragePooling2D
jaxmao_avgpool = jaxmao.layers.AveragePooling2D(kernel_size=(2, 2), strides=(2, 2), padding='SAME')
jaxmao_avgpool.init_params(key)
jaxmao_avg_pooled, state = jaxmao_avgpool(jaxmao_avgpool.params, sample_image)

# TensorFlow AveragePooling2D
tf_avgpool = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
tf_avg_pooled = tf_avgpool(sample_image).numpy()

# Compare
print(close_enough(jaxmao_avg_pooled, tf_avg_pooled).all())

True


# GlobalAveragePooling2d

In [ ]:
# Generate random image data
sample_image = np.random.normal(0, 1, (421, 42, 8, 3))  # shape (batch_size, height, width, channels)

# JaxMao GlobalAveragePooling2D
jaxmao_global_avgpool = jaxmao.layers.GlobalAveragePooling2D()
jaxmao_global_avgpool.init_params(key)  # If initialization is required
jaxmao_global_avg_pooled, state = jaxmao_global_avgpool(jaxmao_global_avgpool.params, sample_image)

# TensorFlow GlobalAveragePooling2D
tf_global_avgpool = keras.layers.GlobalAveragePooling2D()
tf_global_avg_pooled = tf_global_avgpool(sample_image).numpy()

# Compare
print(close_enough(jaxmao_global_avg_pooled, tf_global_avg_pooled).all())

True


# GlobalMaxPooling2d

In [ ]:
# Generate random image data
sample_image = np.random.normal(0, 1, (421, 42, 8, 3))  # shape (batch_size, height, width, channels)

# JaxMao GlobalAveragePooling2D
jaxmao_global_maxpool = jaxmao.layers.GlobalMaxPooling2D()
jaxmao_global_maxpool.init_params(key)  # If initialization is required
jaxmao_global_avg_pooled, state = jaxmao_global_maxpool(jaxmao_global_maxpool.params, sample_image)

# TensorFlow GlobalAveragePooling2D
tf_global_maxpool = keras.layers.GlobalMaxPooling2D()
tf_global_avg_pooled = tf_global_maxpool(sample_image).numpy()

# Compare
print(close_enough(jaxmao_global_avg_pooled, tf_global_avg_pooled).all())

True


# DepthwiseConv2d

In [ ]:
# Generate random image data
sample_image = np.random.normal(0, 1, (5, 8, 8, 42))  # shape (batch_size, height, width, channels)

# JaxMao DepthwiseConv2D
jaxmao_depthwise_conv = jaxmao.layers.DepthwiseConv2D(42, kernel_size=(3, 3), strides=(1, 1), padding='SAME')
jaxmao_depthwise_conv.init_params(key)
jaxmao_depthwise_conv_output, state = jaxmao_depthwise_conv(jaxmao_depthwise_conv.params, sample_image)

# TensorFlow DepthwiseConv2D
tf_depthwise_conv = keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
tf_depthwise_conv.build(input_shape=(None, 8, 8, 42))
tf_depthwise_conv.set_weights([jaxmao_depthwise_conv.params['weights'].reshape(3,3,42,1), jaxmao_depthwise_conv.params['biases']])
tf_depthwise_conv_output = tf_depthwise_conv(sample_image).numpy()

# Compare
print(close_enough(jaxmao_depthwise_conv_output, tf_depthwise_conv_output).all())

True


# Cross entropy

In [ ]:
# Generate sample data
y_true = np.array([0, 1, 2, 2, 1])  # True labels
y_true = jax.nn.one_hot(y_true, num_classes=3).astype('float32')
y_pred = np.array([
    [0.1, 0.2, 0.7],  # Predicted probabilities
    [0.5, 0.4, 0.1],
    [0.02, 0.01, 0.97],
    [0.1, 0.1, 0.8],
    [0.6, 0.3, 0.1]
]).astype('float32')

# JaxMao CrossEntropy
jaxmao_cross_entropy = CategoricalCrossEntropy(reduce_fn='mean_over_batch_size')
jaxmao_loss = jaxmao_cross_entropy.calculate_loss(y_pred, y_true)

# TensorFlow CrossEntropy
tf_cross_entropy = keras.losses.CategoricalCrossentropy()
tf_loss = tf_cross_entropy(y_true, y_pred).numpy()

# Compare
print(close_enough(jaxmao_loss, tf_loss))
jaxmao_loss, tf_loss

True


(Array(0.93529034, dtype=float32), 0.93529034)

In [ ]:
# Generate sample data
y_true = np.array([0, 1, 2, 2, 1])  # True labels
y_true = jax.nn.one_hot(y_true, num_classes=3).astype('float32')
y_pred = np.array([
    [0.1, 0.2, 0.7],  # Predicted probabilities
    [0.5, 0.4, 0.1],
    [0.02, 0.01, 0.97],
    [0.1, 0.1, 0.8],
    [0.6, 0.3, 0.1]
]).astype('float32')

# JaxMao CrossEntropy
jaxmao_cross_entropy = CategoricalCrossEntropy(reduce_fn=jnp.sum)
jaxmao_loss = jaxmao_cross_entropy.calculate_loss(y_pred, y_true)

# TensorFlow CrossEntropy
tf_cross_entropy = keras.losses.CategoricalCrossentropy(reduction='sum')
tf_loss = tf_cross_entropy(y_true, y_pred).numpy()

# Compare
print(close_enough(jaxmao_loss, tf_loss))
jaxmao_loss, tf_loss

True


(Array(4.6764517, dtype=float32), 4.6764517)